In [1]:
#pip install ipython-sql
#pip install pandas_bokeh

In [2]:
import numpy as np
import pandas as pd
import sqlite3
%load_ext sql

import pandas_bokeh
from bokeh.io import show,output_notebook,output_file
from bokeh.plotting import figure
from bokeh.layouts import row
from bokeh.models import Range1d

pandas_bokeh.output_notebook()
pd.set_option("plotting.backend", "pandas_bokeh")

Loading BokehJS ...

IMPORT DATA

In [3]:
df = pd.read_csv("ideal.csv")
cnn = sqlite3.connect("database.db")
df.to_sql("ideal",cnn,if_exists='replace', index=False)
%sql sqlite:///database.db
df1 = %sql SELECT * FROM ideal
ideal_data = pd.DataFrame(df1)

 * sqlite:///database.db
Done.


In [4]:
df = pd.read_csv("train.csv")
cnn = sqlite3.connect("database.db")
df.to_sql("train",cnn,if_exists='replace', index=False)
%sql sqlite:///database.db
df2 = %sql SELECT * FROM train
train_data = pd.DataFrame(df2)


 * sqlite:///database.db
Done.


In [5]:
test_data = pd.read_csv("test.csv")

INITIAL VISUALIZATION

In [6]:
class Visualization:
    def __init__(self,x,y,size,title):
        self.x = x
        self.y = y
        self.size = size
        self.title = title

    def train(self):
        visual = figure(width=280, height=250,title=self.title)
        visual.circle(self.x,self.y,size=self.size)
        return visual

class ChangeRange(Visualization):
    def __init__(self,x,y,size,title,range1,range2):
        super().__init__(x,y,size,title)
        self.range1 = range1
        self.range2 = range2
    
    def changerange(self):
        visual = figure(width=280, height=250,title=self.title)
        visual.circle(self.x,self.y,size=self.size)
        visual.y_range = Range1d(self.range1, self.range2)
        if self.range1 == self.range2:
            raise ValueError("Both of the values for y_range are the same! Please check again")
        else:
            visual.y_range = Range1d(self.range1, self.range2)
        return visual

s1 = Visualization(train_data.iloc[:,0], train_data.iloc[:,1],4,"scatter train_y1")
s2 = Visualization(train_data.iloc[:,0], train_data.iloc[:,2],4,"scatter train_y2")
s3 = ChangeRange(train_data.iloc[:,0], train_data.iloc[:,3],4,"scatter train_y3",-50,50)
s4 = Visualization(train_data.iloc[:,0], train_data.iloc[:,4],4,"scatter train_y4")
s5 = Visualization(test_data.iloc[:,0], test_data.iloc[:,1],4,"scatter test")

show(row(s1.train(),s2.train(),s3.changerange(),s4.train(),s5.train()))


FIND Sum of Squares=∑(y -ŷ)^2 

In [7]:
def ssq(data):
    sums = []
    for num in range(1,51):
        diff = ideal_data.iloc[:,num] - data
        sum_of_square=(np.square(diff)).sum()
        sums.append([sum_of_square])
        df = pd.DataFrame(sums)
        df.index = df.index + 1
        df.columns = ["min sum of square"]
        df_sorted = df.sort_values(by="min sum of square")
    return df_sorted

ssq_df= []
for i in range(1,5):
    result = ssq(train_data.iloc[:,i])
    new_df= result[0:1].round(1)
    new_df["train func"] = "y"+str(i)
    new_df.index.name = "matching ideal func"
    new_df = new_df[["train func","min sum of square"]]
    ssq_df.append(new_df)

ssq_df = pd.concat(ssq_df)
ssq_df

,train func,min sum of square
matching ideal func,,
29,y1,30.2
40,y2,33.2
44,y3,34.4
22,y4,33.5


MATCHING DATA VISUALIZATION

In [8]:
class MatchingDataVisualization(Visualization):
    def __init__(self,x,y,size,title,trainlegend,idealx,idealy,idealsize,ideallegend,location):
        super().__init__(x,y,size,title)
        self.trainlegend = trainlegend
        self.idealx = idealx
        self.idealy = idealy
        self.idealsize = idealsize
        self.ideallegend = ideallegend
        self.location = location
    
    def matching(self):
        visual = figure(width=350, height=300,title=self.title)
        visual.circle(self.x,self.y,size=self.size,legend_label=self.trainlegend)
        visual.circle(self.idealx,self.idealy,size=self.idealsize,color="orange",legend_label=self.ideallegend)
        visual.legend.location = self.location
        return visual

class NewRange(MatchingDataVisualization):
    def __init__(self,x,y,size,title,trainlegend,idealx,idealy,idealsize,ideallegend,location,y_range1,y_range2):
        super().__init__(x,y,size,title,trainlegend,idealx,idealy,idealsize,ideallegend,location)
        self.y_range1 = y_range1
        self.y_range2 = y_range2 

    def newrange(self):
        visual = figure(width=350, height=300,title=self.title)
        visual.circle(self.x,self.y,size=self.size,legend_label=self.trainlegend)
        visual.circle(self.idealx,self.idealy,size=self.idealsize,color="orange",legend_label=self.ideallegend)
        visual.legend.location = self.location
        if self.y_range1 == self.y_range2:
            raise ValueError("Both of the values for y_range are the same! Please check again")
        else:
            visual.y_range = Range1d(self.y_range1, self.y_range2)
        return visual

v1 = MatchingDataVisualization(train_data.iloc[:,0], train_data.iloc[:,1],6,"train_y1 matches ideal_y"+str(ssq_df.index[0]),"train y_1",\
    ideal_data.iloc[:,0],ideal_data["y"+str(ssq_df.index[0])],2,"ideal y_"+str(ssq_df.index[0]),"top_left")
v2 = MatchingDataVisualization(train_data.iloc[:,0], train_data.iloc[:,2],6,"train_y2 matches ideal_y"+str(ssq_df.index[1]),"train y_2",\
    ideal_data.iloc[:,0],ideal_data["y"+str(ssq_df.index[1])],2,"ideal y_"+str(ssq_df.index[1]),"top_right")
v3 = NewRange(train_data.iloc[:,0], train_data.iloc[:,3],6,"train_y3 matches ideal_y"+str(ssq_df.index[2]),"train y_3",\
    ideal_data.iloc[:,0],ideal_data["y"+str(ssq_df.index[2])],2,"ideal y_"+str(ssq_df.index[2]),"top_right",-50,50)
v4 = MatchingDataVisualization(train_data.iloc[:,0], train_data.iloc[:,4],6,"train_y4 matches ideal_y"+str(ssq_df.index[3]),"train y_4",\
    ideal_data.iloc[:,0],ideal_data["y"+str(ssq_df.index[3])],2,"ideal y_"+str(ssq_df.index[3]),"top_center")

show(row(v1.matching(),v2.matching(),v3.newrange(),v4.matching()))
        

TEST DATA MAPPING

In [9]:
new_test_df= []
for xva in test_data.iloc[:,0]:
    values = ideal_data.loc[ideal_data.iloc[:,0] == xva]
    new_test_df.append(values)

new_test_df = pd.concat(new_test_df, ignore_index=True)
new_test_df.insert (1, "y", test_data.iloc[:,1])
selected_df_one = new_test_df[["x","y","y"+str(ssq_df.index[0]),"y"+str(ssq_df.index[1]),"y"+str(ssq_df.index[2]),"y"+str(ssq_df.index[3])]]


selected_df = selected_df_one.copy()

selected_df["diff1"] = abs(selected_df.iloc[:,1] - selected_df.iloc[:,2])
selected_df["diff2"] = abs(selected_df.iloc[:,1] - selected_df.iloc[:,3])
selected_df["diff3"] = abs(selected_df.iloc[:,1] - selected_df.iloc[:,4])
selected_df["diff4"] = abs(selected_df.iloc[:,1] - selected_df.iloc[:,5])

selected_df["No. of ideal func"] = ""

In [10]:
a=(abs(train_data.iloc[:,1]-ideal_data["y"+str(ssq_df.index[0])])).max()*(np.sqrt(2))
b=(abs(train_data.iloc[:,2]-ideal_data["y"+str(ssq_df.index[1])])).max()*(np.sqrt(2))
c=(abs(train_data.iloc[:,3]-ideal_data["y"+str(ssq_df.index[2])])).max()*(np.sqrt(2))
d=(abs(train_data.iloc[:,4]-ideal_data["y"+str(ssq_df.index[3])])).max()*(np.sqrt(2))

class Map:
    def __init__(self,data,criterion,funcname,delta):
        self.data = data
        self.criterion = criterion
        self.funcname = funcname
        self.delta = delta

    def mapping(self):
        required_df=self.data.loc[self.data.iloc[:,2] < self.criterion]
        required_df_copy=required_df.copy()
        required_df_copy["No. of ideal func"] = self.funcname
        rename = required_df_copy.rename(columns = {self.delta : "Delta"})
        return rename

ymap1 = Map(selected_df.iloc[:, [0,1,6,10]],a,"y"+str(ssq_df.index[0]),"diff1")
ymap2= Map(selected_df.iloc[:, [0,1,7,10]],b,"y"+str(ssq_df.index[1]),"diff2")
ymap3 = Map(selected_df.iloc[:, [0,1,8,10]],c,"y"+str(ssq_df.index[2]),"diff3")
ymap4 = Map(selected_df.iloc[:, [0,1,9,10]],d,"y"+str(ssq_df.index[3]),"diff4")

frames = [ymap1.mapping(),ymap2.mapping(),ymap3.mapping(),ymap4.mapping()]
mapped_data = pd.concat(frames)
mapped_data.sort_index(inplace=True)
mapped_data.describe()

,x,y,Delta
count,32.000000,32.000000,32.000000
mean,1.284375,674.887658,0.342820
std,10.131048,2008.151121,0.205162
min,-16.400000,-2838.562500,0.017960
25%,-5.750000,-0.117349,0.120814
50%,0.100000,21.432973,0.401995
75%,9.700000,437.136710,0.530782
max,19.700000,8033.033000,0.637400


EXPORT TO DATABASE

In [11]:
cnn = sqlite3.connect("database.db")
cur = cnn.cursor()
mapped_data.to_sql("test",cnn,if_exists='replace', index=False)
%sql sqlite:///database.db


In [12]:
t1 = figure(width=600, height=400,title="mapped data points")
t1.circle(test_data.iloc[:,0], test_data.iloc[:,1], size=4,legend_label="test_data")
t1.circle(ymap1.mapping().iloc[:,0], ymap1.mapping().iloc[:,1], size=6,color="red",legend_label="y"+str(ssq_df.index[0]))
t1.circle(ymap2.mapping().iloc[:,0], ymap2.mapping().iloc[:,1], size=6,color="orange",legend_label="y"+str(ssq_df.index[1]))
t1.circle(ymap3.mapping().iloc[:,0], ymap3.mapping().iloc[:,1], size=6,color="purple",legend_label="y"+str(ssq_df.index[2]))
t1.circle(ymap4.mapping().iloc[:,0], ymap4.mapping().iloc[:,1], size=6,color="black",legend_label="y"+str(ssq_df.index[3]))
t1.legend.location = "bottom_right"
show(t1)

